In [ ]:
from __future__ import print_function
from __future__ import division
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#filepath=os.path.abspath(os.getcwd())+"\\feature_matrix\\Combined Data CSV - Corrected.csv" # Code assumes it is found in Postural_sway_measures / Change name of csv file as needed
filepath=os.path.abspath("/content/Combined Data CSV - Corrected.csv")
data_diagnosis=pd.read_csv(filepath,sep=',')
data_diagnosis['sex']=(data_diagnosis['sex'] == 'male').astype(int) # 1 for male and 0 for female

#filepath=os.path.abspath(os.getcwd())+"\\feature_matrix\\Full Calculated Feature Matrix - Corrected.csv"
filepath=os.path.abspath("/content/Full Calculated Feature Matrix - Corrected.csv")
data_features=pd.read_csv(filepath,sep=',')
for i in range(len(data_features.index)):
    data_features.iloc[i,0]=(data_features.iloc[i,0]).rstrip('.xlsx')
    
data=data_diagnosis.set_index('name').join(data_features.set_index('FileName'))
data

,sex,age,disease name,layer 1,layer 2,layer 3,Open eye pressure center position (left and right),Center position of the eye pressure (front and back),Eye circumference area,Eye opening effective value area,Total eye-open path length,Eye opening unit area locus length,Eye opening average speed,Closed foot pressure center position (left and right),Eye pressure center position (front and back),Peripheral area with eyes closed,Eye closing effective value area,Total eye-closure length,Eye-closing unit area locus length,Eye closing average speed,Romberg rate (peripheral area),Romberg rate (total track length),Romberg rate (average speed),EO_MDISTrd,EO_MDISTap,EO_MDISTml,EO_sdrd,EO_sdap,EO_sdml,EO_MVELOrd,EO_MVELOap,EO_MVELOml,EO_TOTEXrd,EO_TOTEXap,EO_TOTEXml,EO_RANGErd,EO_RANGEap,EO_RANGEml,EO_AREACC,EO_AREACE,...,EC_AREACE,EC_AREASW,EC_MFREQrd,EC_MFREQap,EC_MFREQml,EC_u0rd,EC_u0ap,EC_u0ml,EC_PF50rd,EC_PF50ap,EC_PF50ml,EC_PF95rd,EC_PF95ap,EC_PF95ml,EC_CFREQrd,EC_CFREQap,EC_CFREQml,EC_FREQDrd,EC_FREQDap,EC_FREQDml,EC_Dcrd,EC_Dcap,EC_Dcml,EC_Dsrd,EC_Dsap,EC_Dsml,EC_Dlrd,EC_Dlap,EC_Dlml,EC_Hcrd,EC_Hcap,EC_Hcml,EC_Hsrd,EC_Hsap,EC_Hsml,EC_Hlrd,EC_Hlap,EC_Hlml,EC_meanCOPy,EC_meanCOPx
AIpt1,1,63,Meniere's disease,2,3,5,-0.301946,-2.436221,3.678812,1.765198,82.694026,22.478459,1.376456,-0.470896,-1.289913,3.315560,1.038198,111.146584,33.522720,1.849550,0.901258,1.344070,1.343705,0.674816,0.486634,0.369216,0.749692,0.587125,0.466179,1.456442,0.738457,1.104767,87.386547,44.307400,66.286028,1.683753,2.737110,2.623690,4.615277,4.906130,...,3.041698,0.334591,0.610298,0.565525,0.793015,1.417379,2.512815,3.616743,0.488281,0.195312,0.292969,2.246094,1.611328,1.855469,1.057345,0.711999,0.821888,0.689870,0.728392,0.697686,0.706427,1.168968,0.906493,0.115831,0.136124,0.195037,-0.000971,-0.004397,-0.001669,-0.256464,-0.088069,-0.193956,0.710533,0.794348,0.750105,-0.025332,-0.088030,-0.026816,-1.289900,-0.470905
AIpt10,1,43,Upper semicircular canal space syndrome (suspe...,2,3,5,1.110603,-2.546055,5.525864,2.931303,83.549253,15.119673,1.390455,1.256180,-1.205303,11.518841,5.787157,105.896299,9.193312,1.763266,2.084532,1.267472,1.268122,0.855625,0.723274,0.352117,0.966077,0.846012,0.466442,1.472349,0.833961,1.046828,88.340951,50.037650,62.809664,2.244171,3.712630,3.308410,7.977416,7.170633,...,15.232132,0.605617,0.257740,0.347809,0.257362,5.365304,3.984451,8.195656,0.341797,0.341797,0.195312,1.025391,0.976562,0.781250,0.546794,0.513483,0.451592,0.616672,0.600779,0.622964,1.741466,1.778399,3.092374,0.376275,0.267692,0.405764,-0.016234,-0.007871,0.132531,-0.013186,-0.010877,-0.050092,0.886429,0.898384,0.902409,-0.055991,-0.037583,0.252131,-1.205289,1.256187
AIpt100,1,49,Lateral medullary syndrome,2,3,6,0.144791,-1.899829,5.564807,2.115183,84.355511,15.158750,1.404680,-0.173170,-1.272717,10.427815,3.219486,153.525659,14.722706,2.556891,1.873886,1.819984,1.820266,0.728134,0.454742,0.477888,0.820643,0.564570,0.595581,1.481947,0.804215,1.086154,88.916810,48.252917,65.169224,2.110254,2.736576,3.675780,5.732321,6.174128,...,9.550803,0.817761,0.475076,0.428772,0.641639,3.698824,6.478350,11.275901,0.585938,0.341797,0.390625,1.855469,1.123047,1.220703,0.901547,0.568117,0.620642,0.616057,0.602526,0.584404,0.539023,0.903116,0.727299,0.363880,0.428487,0.733798,0.007865,0.037355,-0.010226,-0.432793,-0.346515,-0.230871,0.791349,0.869788,0.853686,0.092514,0.197002,-0.053835,-1.272709,-0.173224
AIpt101,1,49,Lateral medullary syndrome,2,3,6,-1.256492,-2.176461,5.858314,2.380948,86.826209,14.821023,1.445719,-0.510439,-1.914177,8.786140,3.563550,159.638441,18.169348,2.657964,1.499773,1.838597,1.838507,0.767647,0.505672,0.473896,0.870689,0.650074,0.579227,1.570134,0.820740,1.157490,94.208038,49.244387,69.449386,1.971103,3.874186,2.845317,6.546438,7.049745,...,10.101598,0.795504,0.458610,0.475588,0.511202,5.243920,7.249965,9.783500,0.390625,0.341797,0.292969,1.464844,0.976562,1.318359,0.767950,0.564170,0.610649,0.663723,0.642452,0.670012,0.565358,0.810213,1.195019,0.419156,0.3

# PREDICTORS (ONE-HOT ENCODING)

In [ ]:
# All layers in one (Value can be 1/4/5/6)
data.insert(6,"all layers",data['layer 3'])

# Layers 4: 1 if non-AB Central-Vestibular
data.insert(6,"layer 4",data['layer 3'])
for i in range(len(data)):
    if(data['layer 4'][i]==1 or data['layer 3'][i]==4):
        data['layer 4'][i]=0
    if(data['layer 4'][i]==5):
        data['layer 4'][i]=0
    if(data['layer 4'][i]==6):
        data['layer 4'][i]=1

# layer 1: 1 if AB 
data['layer 1']=(data['layer 1'] == 1).astype(int) 

# layer 2: 1 if non-AB non-Vestibular
for i in range(len(data)):
    if(data['layer 2'][i]==1):
        data['layer 2'][i]=0
    if(data['layer 2'][i]==3):
        data['layer 2'][i]=0
    if(data['layer 2'][i]==4):
        data['layer 2'][i]=1

# layer 3: 1 if non-AB Peripheral-Vestibular
for i in range(len(data)):
    if(data['layer 3'][i]==1 or data['layer 3'][i]==4):
        data['layer 3'][i]=0
    if(data['layer 3'][i]==5):
        data['layer 3'][i]=1
    if(data['layer 3'][i]==6):
        data['layer 3'][i]=0

## Compiling results
final_results=[]

## Features
first_calculated_feature=data.columns.get_loc("EO_MDISTrd")
first_given_feature=data.columns.get_loc("Open eye pressure center position (left and right)")
# all (calculated) features, not including the diagnosis / sex
all_features=list(data.columns[1:2])+list(data.columns[first_calculated_feature:])
features=all_features[:]
data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-pa

,sex,age,disease name,layer 1,layer 2,layer 3,layer 4,all layers,Open eye pressure center position (left and right),Center position of the eye pressure (front and back),Eye circumference area,Eye opening effective value area,Total eye-open path length,Eye opening unit area locus length,Eye opening average speed,Closed foot pressure center position (left and right),Eye pressure center position (front and back),Peripheral area with eyes closed,Eye closing effective value area,Total eye-closure length,Eye-closing unit area locus length,Eye closing average speed,Romberg rate (peripheral area),Romberg rate (total track length),Romberg rate (average speed),EO_MDISTrd,EO_MDISTap,EO_MDISTml,EO_sdrd,EO_sdap,EO_sdml,EO_MVELOrd,EO_MVELOap,EO_MVELOml,EO_TOTEXrd,EO_TOTEXap,EO_TOTEXml,EO_RANGErd,EO_RANGEap,EO_RANGEml,...,EC_AREACE,EC_AREASW,EC_MFREQrd,EC_MFREQap,EC_MFREQml,EC_u0rd,EC_u0ap,EC_u0ml,EC_PF50rd,EC_PF50ap,EC_PF50ml,EC_PF95rd,EC_PF95ap,EC_PF95ml,EC_CFREQrd,EC_CFREQap,EC_CFREQml,EC_FREQDrd,EC_FREQDap,EC_FREQDml,EC_Dcrd,EC_Dcap,EC_Dcml,EC_Dsrd,EC_Dsap,EC_Dsml,EC_Dlrd,EC_Dlap,EC_Dlml,EC_Hcrd,EC_Hcap,EC_Hcml,EC_Hsrd,EC_Hsap,EC_Hsml,EC_Hlrd,EC_Hlap,EC_Hlml,EC_meanCOPy,EC_meanCOPx
AIpt1,1,63,Meniere's disease,0,0,1,0,5,-0.301946,-2.436221,3.678812,1.765198,82.694026,22.478459,1.376456,-0.470896,-1.289913,3.315560,1.038198,111.146584,33.522720,1.849550,0.901258,1.344070,1.343705,0.674816,0.486634,0.369216,0.749692,0.587125,0.466179,1.456442,0.738457,1.104767,87.386547,44.307400,66.286028,1.683753,2.737110,2.623690,...,3.041698,0.334591,0.610298,0.565525,0.793015,1.417379,2.512815,3.616743,0.488281,0.195312,0.292969,2.246094,1.611328,1.855469,1.057345,0.711999,0.821888,0.689870,0.728392,0.697686,0.706427,1.168968,0.906493,0.115831,0.136124,0.195037,-0.000971,-0.004397,-0.001669,-0.256464,-0.088069,-0.193956,0.710533,0.794348,0.750105,-0.025332,-0.088030,-0.026816,-1.289900,-0.470905
AIpt10,1,43,Upper semicircular canal space syndrome (suspe...,0,0,1,0,5,1.110603,-2.546055,5.525864,2.931303,83.549253,15.119673,1.390455,1.256180,-1.205303,11.518841,5.787157,105.896299,9.193312,1.763266,2.084532,1.267472,1.268122,0.855625,0.723274,0.352117,0.966077,0.846012,0.466442,1.472349,0.833961,1.046828,88.340951,50.037650,62.809664,2.244171,3.712630,3.308410,...,15.232132,0.605617,0.257740,0.347809,0.257362,5.365304,3.984451,8.195656,0.341797,0.341797,0.195312,1.025391,0.976562,0.781250,0.546794,0.513483,0.451592,0.616672,0.600779,0.622964,1.741466,1.778399,3.092374,0.376275,0.267692,0.405764,-0.016234,-0.007871,0.132531,-0.013186,-0.010877,-0.050092,0.886429,0.898384,0.902409,-0.055991,-0.037583,0.252131,-1.205289,1.256187
AIpt100,1,49,Lateral medullary syndrome,0,0,0,1,6,0.144791,-1.899829,5.564807,2.115183,84.355511,15.158750,1.404680,-0.173170,-1.272717,10.427815,3.219486,153.525659,14.722706,2.556891,1.873886,1.819984,1.820266,0.728134,0.454742,0.477888,0.820643,0.564570,0.595581,1.481947,0.804215,1.086154,88.916810,48.252917,65.169224,2.110254,2.736576,3.675780,...,9.550803,0.817761,0.475076,0.428772,0.641639,3.698824,6.478350,11.275901,0.585938,0.341797,0.390625,1.855469,1.123047,1.220703,0.901547,0.568117,0.620642,0.616057,0.602526,0.584404,0.539023,0.903116,0.727299,0.363880,0.428487,0.733798,0.007865,0.037355,-0.010226,-0.432793,-0.346515,-0.230871,0.791349,0.869788,0.853686,0.092514,0.197002,-0.053835,-1.272709,-0.173224
AIpt101,1,49,Lateral medullary syndrome,0,0,0,1,6,-1.256492,-2.176461,5.858314,2.380948,86.826209,14.821023,1.445719,-0.510439,-1.914177,8.786140,3.563550,159.638441,18.169348,2.657964,1.499773,1.838597,1.838507,0.767647,0.505672,0.473896,0.870689,0.650074,0.579227,1.570134,0.820740,1.157490,94.208038,49.244387,69.449386,1.971103,3.874186,2.845317,...,10.101598,0.795504,0.458610,0.475588,0.511202,5.243920,7.249965,9.783500,0.390625,0.341797,0.292969,1.464844,0.976562,1.318359,0.767950,0.564170,0.610649,0.663723,0.642452,0.670012,0.565358,0.810213,1.195019,0.419156,0.398222,0.693313,0.002114,0.008877,-0.022853,-0.357479,-0.2

# TRAIN-VALIDATION-TEST SPLIT

In [ ]:
X_data = data[features]
Y_data = data[list(data.columns[3:7])] #['layer 1', 'layer 2', 'layer 3', 'layers 1+2']

np.random.seed(1)
N = X_data.index.values
np.random.shuffle(N)
X_data = (X_data.reindex(N)).values
Y_data = (Y_data.reindex(N)).values

#Next, we partition the randomly shuffled dataset into training, validation and testset according a ratio
train_ratio = 0.80
valid_ratio = 0.1
n_train = int(len(X_data) * train_ratio)
n_valid = int(len(X_data) * valid_ratio)

X_train, X_valid, X_test = X_data[:n_train], X_data[n_train:n_train+n_valid], X_data[n_train+n_valid:]
Y_train, Y_valid, Y_test = Y_data[:n_train], Y_data[n_train:n_train+n_valid], Y_data[n_train+n_valid:]

# NEURAL NETWORK

In [ ]:
#Install Objax
!pip --quiet install  objax
import objax

     |████████████████████████████████| 51kB 4.1MB/s 


In [ ]:
import tensorflow as tf 
import tensorflow_datasets as tfds
import numpy as np
import jax.numpy as jn
import random 
import matplotlib.pyplot as plt

In [ ]:
class NeuralNet(objax.Module):
  def __init__(self, number_of_channels = len(features), number_of_classes = 4):
    self.number_of_channels = number_of_channels
    self.number_of_classes = number_of_classes
    self.MyModel = objax.nn.Sequential([objax.nn.Linear(self.number_of_channels, 16), objax.functional.relu,
                                        objax.nn.Linear(16, 16), objax.functional.relu,
                                        objax.nn.Linear(16, self.number_of_classes)
                                        ])
  
  def __call__(self, x):
    return self.MyModel(x)

#The following line creates the Neural Network
model = NeuralNet()
print(model.vars())

(NeuralNet).MyModel(Sequential)[0](Linear).b       16 (16,)
(NeuralNet).MyModel(Sequential)[0](Linear).w     1808 (113, 16)
(NeuralNet).MyModel(Sequential)[2](Linear).b       16 (16,)
(NeuralNet).MyModel(Sequential)[2](Linear).w      256 (16, 16)
(NeuralNet).MyModel(Sequential)[4](Linear).b        4 (4,)
(NeuralNet).MyModel(Sequential)[4](Linear).w       64 (16, 4)
+Total(6)                                        2164


In [ ]:
#Loss function as averaged value of of cross entropies
def loss_function(x, labels):
    logit = model(x)
    return objax.functional.loss.cross_entropy_logits_sparse(logit, labels).mean()

#Prediction function
predict = objax.Jit(lambda x: objax.functional.softmax(model(x)), model.vars()) 

#Calculate the gradient and value of loss_function
gv= objax.GradValues(loss_function, model.vars())

#Provide trainable variables in the model
tv = objax.ModuleList(objax.TrainRef(x) for x in model.vars().subset(objax.TrainVar))

#Training routine
def train_op(x, y, learning_rate):
    lr = learning_rate
    gradient, loss_value = gv(x, y)   # calculate gradient and loss value "backprop"
    #next we update the trainable parameter using SGD and similar procedure
    for grad, params in zip(gradient, tv.vars()):
      params.value=params.value-(lr*grad)                      
    return loss_value   # return loss value

#Make train_op (much) faster using JIT compilation
train_op = objax.Jit(train_op, gv.vars() + tv.vars())


# Training Function (Stochastic Gradient Descent)
def train(EPOCHS = 40, BATCH = 32, LEARNING_RATE = 9e-4):
  avg_train_loss_epoch = []
  avg_val_loss_epoch = []
  train_acc_epoch = []
  val_acc_epoch = []

  for epoch in range(EPOCHS):
      avg_train_loss = 0 # (averaged) training loss per batch
      avg_val_loss =  0  # (averaged) validation loss per batch
      train_acc = 0      # training accuracy per batch
      val_acc = 0        # validation accuracy per batch

      # shuffle the examples prior to training to remove correlation 
      train_indices = np.arange(len(X_train)) 
      np.random.shuffle(train_indices)
      for it in range(0, X_train.shape[0], BATCH):
          batch = train_indices[it:(it+BATCH)] 
          avg_train_loss += float(train_op(X_train[batch], Y_train[batch], LEARNING_RATE)[0]) * len(batch)
          train_prediction = predict(X_train[batch]).argmax(1)
          train_acc += (np.array(train_prediction).flatten() == Y_train[batch]).sum()
      train_acc_epoch.append(train_acc/X_train.shape[0])
      avg_train_loss_epoch.append(avg_train_loss/X_train.shape[0])

      # run validation
      val_indices = np.arange(len(X_valid)) 
      np.random.shuffle(val_indices)    
      for it in range(0, X_valid.shape[0], BATCH):
          batch = val_indices[it:(it+BATCH)] 
          avg_val_loss += float(loss_function(X_valid[batch], Y_valid[batch])) * len(batch)
          val_prediction = predict(X_valid[batch]).argmax(1)
          val_acc += (np.array(val_prediction).flatten() == Y_valid[batch]).sum()
      val_acc_epoch.append(val_acc/X_valid.shape[0])
      avg_val_loss_epoch.append(avg_val_loss/X_valid.shape[0])

      print('Epoch %04d  Training Loss %.2f Validation Loss %.2f Training Accuracy %.2f Validation Accuracy %.2f' % (epoch + 1, avg_train_loss/X_train.shape[0], avg_val_loss/X_valid.shape[0], 100*train_acc/X_train.shape[0], 100*val_acc/X_valid.shape[0]))
  
  #Plot training loss
  plt.title("Train vs Validation Loss")
  plt.plot(avg_train_loss_epoch, label="Train")
  plt.plot(avg_val_loss_epoch, label="Validation")
  plt.xlabel("Epoch")
  plt.ylabel("Loss")
  plt.legend(loc='best')
  plt.show()

  plt.title("Train vs Validation Accuracy")
  plt.plot(train_acc_epoch, label="Train")
  plt.plot(val_acc_epoch, label="Validation")
  plt.xlabel("Epoch")
  plt.ylabel("Accuracy (%)")
  plt.legend(loc='best')
  plt.show()

In [ ]:
train()

ValueError: ignored